In [ ]:
#importing libraires
import numpy as np
import geemap
import pandas as pd
import os
import geopandas as gpd
import folium
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler
import ee 


In [ ]:
#triggering the authentication flow
ee.Authenticate()

#intialize the library
ee.Initialize()

# initializes the Google Earth Engine API
geemap.ee_initialize()

Define Functions for customised visualzation.

In [ ]:
# Define a function for visualizing a single vector layer
def vectorMap(layer, viz_params, name):

    """

    """
    map = geemap.Map(center=[12.422777,10.028956], zoom=13)
    map.add_basemap("SATELLITE")
    map.addLayer(layer, vis_params=viz_params, name=name, opacity=0.5)
    return map

In [ ]:
# Define a function for visualizing a single raster layer
def rasterMap(layer, viz_params, name):

    """

    """
    map = geemap.Map(center=[12.422777,10.028956], zoom=13)
    map.add_basemap("SATELLITE")
    map.addLayer(layer, vis_params=viz_params, name=name)
    return map

In [ ]:
# Define a function for visualizing four (4) layers using a linked map 
def linkedMap(layers, viz_params, labels):
    """

    """

    map = geemap.linked_maps(
     rows=2,
     cols=2,
        height="450px",
        center=[12.422777,10.028956],
        zoom=13,
        ee_objects=layers,
        vis_params=viz_params,
        labels=labels,
        label_position="topright",
    )
    return map

In [ ]:
# Define a function to visualize two (2) layers using slider Map

def sliderMap(layer1params, layer2params):
    """

    """

    ee_object1, vis_params1, name1 = layer1params
    ee_object2, vis_params2, name2 = layer2params
    
    left_layer = geemap.ee_tile_layer(ee_object1, vis_params1, name=name1)
    right_layer = geemap.ee_tile_layer(ee_object2, vis_params2, name=name2)
    
    map = geemap.Map(center=(12.422777,10.028956), zoom =13)

    map.split_map(left_layer, right_layer)
    return map

Data Downlaoding and processing: Boundary Data

In [ ]:
# Load the boundary of Haejia
adm2 = ee.FeatureCollection("FAO/GAUL/2015/level2")
hadejia_bdry = adm2.filter(ee.Filter.eq("ADM2_NAME", "Hadejia"))
roi = hadejia_bdry.geometry()

In [ ]:
# Visualize the boundary
vectorMap(roi, {"fillColor": "00000000", "color": "FF0000"}, "Hadejia Boundary")

Satellite images

In [ ]:
# Set the start and end dates to search for images
start_date = "2023-02-01"
end_date = "2023-11-30"

In [ ]:
# Define a function that download and process time-series sentinel-2 images
def s2Process(dates, extent, cloud):

    """

    """
    start_date, end_date = dates

    # Load and filter Sentinel-2 image collection 
    def imgDownload():
        
        img_collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                    .filterDate(start_date, end_date)
                    .filter(ee.Filter.eq('MGRS_TILE', '32PPU'))
                    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", cloud))
                    .filterBounds(extent))

        print(f"There are {img_collection.size().getInfo()} Sentinel-2 Images")
        return img_collection

# function for cloud masking
    def s2ClearSky(image):
      
        scl = image.select('SCL')
        clear_sky_pixels = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(11))
        return image.updateMask(clear_sky_pixels).multiply(0.0001)
    
    # function for resampling 20 meter bands to 10m
    def bandResample(image):
       
        bands_10m = image.select(["B2", "B3", "B4", "B8"])
        bands_20m = image.select(["B5","B6", "B7", "B8A", "B11", "B12"]).resample("bilinear").reproject(crs=bands_10m.projection(), scale=10)
        return bands_10m.addBands(bands_20m)


    # function to compute spectral indices
    def indices(image):
        
        ndvi = image.normalizedDifference(["B8", "B4"]).rename("ndvi")
        mndwi = image.normalizedDifference(["B3", "B11"]).rename("mndwi")
        ndbi = image.normalizedDifference(["B11", "B8"]).rename("ndbi")
        bsi = image.expression(
         "((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1 + RED) + (NIR + BLUE))",
            {
                "SWIR1": image.select(["B11"]),
                "RED": image.select(["B4"]),
                "NIR": image.select(["B8"]),
                "BLUE": image.select(["B2"]),
            }
        ).rename("bsi")
        return image.addBands([ndvi, mndwi, ndbi, bsi])
    
    # Donwload the image collection
    s2_collection = imgDownload()

    # Apply the functions to remove clouds, resmaple bands, and compute spectral indices
    s2_collection_resampled = (s2_collection
                           .map(s2ClearSky)
                           .map(bandResample)
                           .map(indices))
    
    # Compute the median composite
    s2_median= s2_collection_resampled.median()

    # Clip the median composite image to the region of interest
    s2_median= s2_median.clip(extent)

    # Check the "s2_median" properties

    # Get the projection information from one of the bands
    s2_median_projection = s2_median.select(0).projection()

    # Get the CRS and spatial resolution
    s2_median_crs = s2_median_projection.crs().getInfo()
    s2_median_scale = s2_median_projection.nominalScale().getInfo()

    print(f"The CRS of s2_median is {s2_median_crs}")
    print(f"The spatial resolution of s2_median is {s2_median_scale}")

    # Check the bands in "s2_median"
    s2_band_names = s2_median.bandNames().getInfo()
    print(f"The following are the bands in 's2_median': {s2_band_names}")


    return s2_median

In [ ]:
# Download s2-image collectionn, remove clouds, resmaple bands, compute spectral indices, and median composites
dates = (start_date, end_date)
s2_median_2023 = s2Process(dates, roi, 30)

In [ ]:
# Visualize the "s2_median_2023"
s2_vis_params = {"min":0, "max": 0.3, "bands":["B8", "B4", "B3"], "gamma":0.6}

rasterMap(s2_median_2023, s2_vis_params, "S2 Median Composite 2023")


In [ ]:
# Define a function that download and process time-series sentinel-1 images

def s1Process(dates, extent, radius):
    """
    """
    start_date, end_date = dates

    # Function to load and filter Sentinel-1 image collection 
    def imgdownload():
        img_collection = (ee.ImageCollection("COPERNICUS/S1_GRD")
                .filterDate(start_date, end_date)
                .filter(ee.Filter.eq("instrumentMode", "IW"))
                .filterMetadata("resolution_meters", "equals", 10)
                .filterBounds(extent)
                .select(["VV", "VH"]))

        print(f"There are {img_collection.size().getInfo()} Sentinel-1 Images")

        # Check the available polarizations in the s1_collection
        img_collection_polarizations = img_collection.first().bandNames().getInfo()

        print(f"The following are the polarizations in the Sentinel-1 collection: {img_collection_polarizations}")
        return img_collection

    # Function to apply speckle filter
    def speckle(image):
        image_filtered = image.focal_median(radius, "circle", "meters")
        return image_filtered.copyProperties(image, image.propertyNames())

     # Function to compute VH / VV ratio
    def ratio(image):
        vh = image.select("VH")
        vv = image.select("VV")
        ratio = vh.divide(vv).rename("VH_VV_ratio")
        return image.addBands([ratio])

    # Download the s-1 image collection
    s1_collection = imgdownload()

    # Apply speckle-filter and compute ratio to the S1_collection
    s1_collection_filtered = (s1_collection
                              .map(speckle)
                              .map(ratio))

    # compute median composites
    s1_median =  s1_collection_filtered.median()

    # Clip to the extent
    s1_median = s1_median.clip(extent)


    # Check the "s1_median" properties

    # Get the projection information from one of the bands
    s1_median_projection = s1_median.select(0).projection()

    # Get the CRS and spatial resolution
    s1_median_crs = s1_median_projection.crs().getInfo()
    s1_median_scale = s1_median_projection.nominalScale().getInfo()

    print(f"The CRS of s1_median is {s1_median_crs}")
    print(f"The spatial resolution of s1_median is {s1_median_scale}")

    # Check the bands in "s2_median"
    s1_band_names = s1_median.bandNames().getInfo()
    print(f"The following are the bands in 's1_median': {s1_band_names}")

    return s1_median

In [ ]:
# Download s1-image collectionn, apply speckle filter, compute ratio, and median composites
dates = (start_date, end_date)
s1_median_2023 = s1Process(dates, roi, 50)

# Select different polarizations from s1_median_2023
s1_median_2023_VH = s1_median_2023.select("VH")
s1_median_2023_VV = s1_median_2023.select("VV")
s1_median_2023_ratio = s1_median_2023.select("VH_VV_ratio")

In [ ]:
# Create a linked Map of Sentinel-1 VH, VH, VH_VV_Ratio, and Sentinel-2 median

# Define a list of layers
layers = [s1_median_2023_VH, s1_median_2023_VV, s1_median_2023_ratio, s2_median_2023]

# Define the visualization parameters
linked_viz_params = [
    {"min":-20, "max":-1, "bands":["VH"], "gamma":1.5},
    {"min":-20, "max":-1, "bands":["VV"], "gamma": 1},
    {"min":0, "max":3, "bands":["VH_VV_ratio"], "gamma": 0.95},
    {"min": 0, "max": 0.3, "bands": ["B4", "B3", "B2"], "gamma": 0.95},
]

# Create labels for the link map
linked_labels = [
    "Sentinel-1 (VV Median)",
    "Sentinel-1 (VH Median)",
    "Sentinel-1 (VH_VV Ratio)",
    "Sentinel-2 (Median)"
]

# Visualize the layers
linkedMap(layers, linked_viz_params, linked_labels)

In [ ]:
stacked_layer = s2_median_2023.addBands(s1_median_2023_VH).addBands(s1_median_2023_VV).addBands(s1_median_2023_ratio) 
print(f"The bands in the stacked layer are: {stacked_layer.bandNames().getInfo()}")

In [ ]:
# Check the properties of the "stacked_layer"

# Get the projection information from one of the bands
stacked_layer_projection = stacked_layer.select(16).projection()

# Get the CRS and spatial resolution
stacked_layer_crs = stacked_layer_projection.crs().getInfo()
stacked_layer_scale = stacked_layer_projection.nominalScale().getInfo()

print(f"The CRS of the stacked_layer is {stacked_layer_crs}")
print(f"The spatial resolution of the stacked_layer is {stacked_layer_scale}")

Train and Test

In [ ]:
# Load the training and testing data
sample_gdf = gpd.read_file("Data\Training_Testing_Samples_2023_WGS84.shp")

sample_gdf.info()